In [ ]:

git clone https://github.com/ms-van3t-devs/ms-van3t ms-van3t
cd ms-van3t

git clone https://github.com/nvlabs/sionna sionna-rt

docker build -t sionna-rt:latest sionna-rt/


# save as coexistence_analysis.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# --- تنظیمات ---
k_B = 1.380649e-23  # Boltzmann constant
T = 290.0  # Kelvin
# helper dBm <-> linear
def dbm_to_w(dbm):
    return 10**((dbm-30)/10.0)
def w_to_dbm(w):
    return 10*np.log10(w)+30

# بارگذاری داده شبیه‌سازی (فرمتِ مثال)
df = pd.read_csv("sim_packets.csv", parse_dates=['time'])


def resource_overlap(tx_row, other_row):

    a0,a1 = tx_row['freq_start_Hz'], tx_row['freq_end_Hz']
    b0,b1 = other_row['freq_start_Hz'], other_row['freq_end_Hz']
    return not (a1 <= b0 or b1 <= a0)


grouped = df.groupby(['time','rx_id'])

results = []
Gthr_dB = -120
Gthr_lin = 10**(Gthr_dB/10.0)

for (time, rx), group in grouped:

    group = group.copy()
    group['path_gain_lin'] = 10**(group['path_gain_dB']/10.0)
    # محاسبهٔ received power (W) از tx_power_dBm و path gain:
    group['tx_power_W'] = group['tx_power_dBm'].apply(dbm_to_w)
    group['rx_power_W'] = group['tx_power_W'] * group['path_gain_lin']


    for idx, row in group.iterrows():
        desired = row
        interferers = group.loc[group.index != idx]
        # فیلتر بر اساس همپوشانی ریسورس (باند/ساب‌کاریر)
        interferers = interferers[interferers.apply(lambda r: resource_overlap(desired, r), axis=1)]

        interferers = interferers[interferers['path_gain_lin'] >= Gthr_lin]
        Pint_W = interferers['rx_power_W'].sum() if not interferers.empty else 0.0

        bw = desired['freq_end_Hz'] - desired['freq_start_Hz']
        N0_W = k_B * T * bw
        SINR_lin = desired['rx_power_W'] / (N0_W + Pint_W + 1e-30)
        SINR_dB = 10*np.log10(SINR_lin)
        rssi_dBm_est = w_to_dbm(desired['rx_power_W'])
        results.append({
            'time': time,
            'rx_id': rx,
            'tx_id': desired['tx_id'],
            'technology': desired['technology'],
            'SINR_dB': SINR_dB,
            'PRx_dBm_est': rssi_dBm_est,
            'Pint_dBm': w_to_dbm(Pint_W) if Pint_W>0 else -np.inf
        })

res_df = pd.DataFrame(results)
res_df.to_csv("sim_results_sinr.csv", index=False)
print("Wrote sim_results_sinr.csv with", len(res_df), "rows")

# --- نمودار PDF SINR (نمودار مشابه شکل 7 مقاله) ---
plt.figure(figsize=(8,5))
for tech, g in res_df.groupby('technology'):
    vals = g['SINR_dB'].dropna().values
    if len(vals)>0:
        # KDE ساده: استفاده از histogram نرمال‌شده
        plt.hist(vals, bins=60, density=True, alpha=0.6, label=str(tech))
plt.xlabel("SINR (dB)")
plt.ylabel("PDF (normalized)")
plt.legend()
plt.title("SINR PDF (simulated)")
plt.savefig("sinr_pdf.png", dpi=150)
plt.close()

print("Plotted sinr_pdf.png")

def compute_DR(dfA, dfB):
    # index keys: time,tx_id,rx_id
    key_cols = ['time','tx_id','rx_id']
    A = dfA.set_index(key_cols)
    B = dfB.set_index(key_cols)
    # union indices
    union_idx = A.index.union(B.index)
    A_vals = A.reindex(union_idx)['verdict'].fillna(-1).astype(int)
    B_vals = B.reindex(union_idx)['verdict'].fillna(-1).astype(int)
    sym_diff = (A_vals != B_vals)
    dr = sym_diff.sum() / len(union_idx)
    return dr

# dfA = pd.read_csv("decisions_msvan3t.csv", parse_dates=['time'])
# dfB = pd.read_csv("decisions_van3twin.csv", parse_dates=['time'])
# print("DR:", compute_DR(dfA, dfB))


In [ ]:
{
  "type": "init",
  "scene_id": "turin_center",
  "meshes": [
    {"id":"bldg_1", "path": "/meshes/bldg_1.obj", "material": {"eps_r": 5.31, "sigma": 0.4838}},
    ...
  ],
  "vehicles": [
    {"veh_id":"veh_12", "mesh_id":"car_sedan", "antenna_offset":[0.0,0.0,1.8]}
  ],
  "timestamp": 0
}


In [ ]:
#location update
{
  "type": "loc_update",
  "vehicle_id": "veh_12",
  "pos": [x, y, z],
  "vel": [vx, vy, vz],
  "heading": theta,
  "timestamp": t
}


In [ ]:
#request/response
#reqest
{ "type":"chan_req", "tx_id":"veh_A", "rx_id":"veh_B", "timestamp": t }
#response
{
  "type":"chan_resp",
  "tx_id":"veh_A",
  "rx_id":"veh_B",
  "G_dB": -84.3,
  "tau_s": 1.2e-6,
  "los_flag": 0,
  "paths": [ {"alpha": "...", "delay": "...", "doppler": "...", "AOD": "...", "AOA": "..."} ]
}


In [ ]:
# Initialization
send_init_to_RT(scene_meshes, materials, antenna_offsets)

# Main simulation loop (per simulation time-step or per packet event)
for each time step or packet_event:
    update positions from Mobility Engine (SUMO)
    if position change > Δd_min: send loc_update to RT

    when ms-van3t needs channel info for tx->rx:
        send chan_req(tx, rx, timestamp) to RT
        resp = wait_for_chan_resp()
        if resp from cache or new computation -> get G, paths, los_flag

    # collect all concurrent transmitters to rx for same resource blocks
    interferers = find_overlapping_transmitters(tx, rx, resource_grid)

    # compute rx powers, noise, Pint per block
    for each block z:
        compute SINR_z = Prx_desired_z / (N_z + sum Prx_interferers_z)
    compute weighted average SINR
    determine packet reception via BLER/threshold -> verdict

    record metrics (RSSI, SINR, verdict etc.)
end
